# 大模型的训练流程： PreTrain  --> SFT --> Reward Model --> PPO
PPO  算法需要一个奖励模型来为模型的输出进行打分提供奖励
首先需要收集数据，数据是一个三元组偏好数据{question, chosen, rejected}，其中question是问题，chosen是选择的答案（好的回答），rejected是拒绝的答案（不好的回答）。
奖励模型指导大模型的训练，因此奖励模型的能力要和训练的模型能力一致（因为评价比生成要求低）或者更好

## 损失函数 

### 公式解释

Loss = - logsigmoid(chosen - rejected)
Loss = - log(1 / (1 + e^(-x)))

1. **LogSigmoid 函数**:
   - `logsigmoid(x)` 是对 `sigmoid(x)` 函数取对数。`sigmoid(x)` 函数定义为 `1 / (1 + exp(-x))`，它将输入值 `x` 映射到 (0, 1) 区间，表示某个事件发生的概率。
   - `logsigmoid(x)` 函数则将这个概率映射到实数范围，这在数学上更便于处理，尤其是在优化问题中。

2. **损失函数**:
   - `Loss = - logsigmoid(chosen - rejected)`：这个公式表示损失函数是 `chosen - rejected` 的 logsigmoid 函数的负值。
   - 这里，`chosen` 和 `rejected` 代表两个动作的值，通常是策略网络输出的对数概率（logits）。`chosen` 是被选择的动作的值，而 `rejected` 是被拒绝的动作的值。

3. **等价形式**:
   - `Loss = - log(1 / (1 + e^(-x)))`，其中 `x = chosen - rejected`。
   - 这个等价形式直接展示了 logsigmoid 函数的数学表达式，其中 `x` 是两个动作值的差。

### 应用场景

这种损失函数在强化学习中，特别是在策略梯度方法中，用于优化策略。它鼓励策略网络更倾向于选择那些能够带来更高回报的动作。具体来说：

- 当 `chosen` 的值远大于 `rejected` 的值时，`chosen - rejected` 的值会很大，`logsigmoid(chosen - rejected)` 的值接近 0，因此损失函数的值会很小，这表示策略选择的动作是正确的。
- 相反，如果 `chosen` 和 `rejected` 的值相近，或者 `chosen` 的值小于 `rejected` 的值，损失函数的值会较大，这表示策略需要调整，以更倾向于选择 `chosen` 动作。

### 总结

这个损失函数通过比较被选择的动作和被拒绝的动作的值，来指导策略网络的学习过程。它的目标是最小化损失，从而使得策略网络能够更倾向于选择那些能够带来更高回报的动作。这是强化学习中策略优化的一个关键步骤。


### 训练的一个奖励模型

In [ ]:
import torch
from datasets import Dataset
import json
from peft import LoraConfig, TaskType, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForSequenceClassification
from trl import RewardTrainer, RewardConfig


: 

In [ ]:
model_path = r'D:\work\models\Meta-Llama-3.1-8B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
model = AutoModelForSequenceClassification.from_pretrained(model_path,
                                                           num_labels=1,# 分类为一时是一个回归模型
                                                           quantization_config=bnb_config)
model.config.pad_token_id = tokenizer.pad_token_id
peft_config = LoraConfig(
    r=8,
    target_modules=["q_proj",
                    "v_proj",
                    "k_proj",
                    "o_proj",
                    "gate_proj",
                    "down_proj",
                    "up_proj"
                    ],
    task_type=TaskType.SEQ_CLS,
    lora_alpha=16,
    lora_dropout=0.05
)
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

items = []
with open("./data/preference.json", "r", encoding="utf8") as f:
    for line in f:
        item = json.loads(line)
        items.append(item)

dataset = Dataset.from_list(items)


In [ ]:
def process_func(example):
    chosen = example["question"] + example["chosen"]# 将问题和回答拼接在一起
    rejected = example["question"] + example["rejected"]

    tokenized_chosen = tokenizer(chosen)
    tokenized_rejected = tokenizer(rejected)

    new_example = {}
    new_example["input_ids_chosen"] = tokenized_chosen["input_ids"]
    new_example["attention_mask_chosen"] = tokenized_chosen["attention_mask"]
    new_example["input_ids_rejected"] = tokenized_rejected["input_ids"]
    new_example["attention_mask_rejected"] = tokenized_rejected["attention_mask"]
    return new_example


dataset = dataset.map(process_func, remove_columns=['question', 'chosen', 'rejected'])
print(dataset)

config = RewardConfig(output_dir="./reward_model")
config.num_train_epochs = 1
config.per_device_train_batch_size = 1

trainer = RewardTrainer(
    model=model,
    tokenizer=tokenizer,
    args=config,
    train_dataset=dataset
)
trainer.train()
trainer.save_model("./reward_model")